Imports e Configuração

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Vamos usar Random Forest porque funciona melhor "logo à primeira" que a Classificação Linear pura para futebol
# Mas o conceito de "classes" (0, 1, 2) é o mesmo da tua aula.

sns.set_theme(style="whitegrid")
%matplotlib inline

Carregar Dados Reais (do GitHub)

In [3]:
import os

# File path for the local database
local_file = "premier_league_matches.csv"

# Let's get the last 5 years for a good training set
seasons = ["2425", "2324", "2223", "2122", "2021"]
base_url = "https://www.football-data.co.uk/mmz4281/"

# 1. Check if local file exists
if os.path.exists(local_file):
    print(f"📂 Local file '{local_file}' found. Loading data from disk...")
    df = pd.read_csv(local_file)
    print(f"✅ Data loaded! Total matches: {df.shape[0]}")

# 2. Download from Football-Data.co.uk if missing
else:
    print("🌐 Local file not found. Downloading historical match data...")
    all_seasons_data = []
    
    for season in seasons:
        print(f"   ⬇️ Downloading season {season}...", end=" ")
        try:
            # Football-Data.co.uk structure: base_url/season/LeagueCode.csv
            # E0 = English Premier League
            url = f"{base_url}{season}/E0.csv"
            
            df_season = pd.read_csv(url)
            
            # Add a column for the season
            df_season['Season'] = season
            
            all_seasons_data.append(df_season)
            print(f"Done! ({df_season.shape[0]} matches)")
            
        except Exception as e:
            print(f"\n   ❌ Error downloading {season}: {e}")

    # 3. Merge and Save
    if all_seasons_data:
        df = pd.concat(all_seasons_data, ignore_index=True)
        
        # Convert Date to datetime object (fixes many future errors)
        df['Date'] = pd.to_datetime(df['Date'], dayfirst=True, errors='coerce')
        
        # Save to CSV
        df.to_csv(local_file, index=False)
        print(f"\n💾 Saved to '{local_file}'. Total matches: {df.shape[0]}")
    else:
        print("⚠️ No data found.")

# Show the data structure (HomeTeam, AwayTeam, FTR = Full Time Result, Odds)
df[['Date', 'HomeTeam', 'AwayTeam', 'FTR', 'B365H', 'B365D', 'B365A']].head()

🌐 Local file not found. Downloading historical match data...
   ⬇️ Downloading season 2425... Done! (380 matches)
   ⬇️ Downloading season 2324... Done! (380 matches)
   ⬇️ Downloading season 2223... Done! (380 matches)
   ⬇️ Downloading season 2122... Done! (380 matches)
   ⬇️ Downloading season 2021... Done! (380 matches)

💾 Saved to 'premier_league_matches.csv'. Total matches: 1900


,Date,HomeTeam,AwayTeam,FTR,B365H,B365D,B365A
0,2024-08-16,Man United,Fulham,H,1.60,4.20,5.25
1,2024-08-17,Ipswich,Liverpool,A,8.50,5.50,1.33
2,2024-08-17,Arsenal,Wolves,H,1.18,7.50,13.00
3,2024-08-17,Everton,Brighton,A,2.63,3.30,2.63
4,2024-08-17,Newcastle,Southampton,H,1.36,5.25,8.00


Preparar os Dados (Feature Selection)

Dividir em Treino e Teste

Treinar o Modelo (Linear Regression)

Avaliar e Visualizar